# 🔧 Notebook 2 — Feature Engineering

**Project:** Stock Trend Predictor  
**Goal:** Build and validate all features used for training

### What we cover:
1. Technical indicator computation
2. Feature validation & sanity checks
3. Sentiment feature integration
4. Feature importance preview
5. Final feature matrix assembly

In [ ]:
import sys
sys.path.append('../src')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import yfinance as yf
import ta
import warnings
warnings.filterwarnings('ignore')

from features import add_technical_indicators, create_labels, prepare_feature_matrix
from sentiment import score_headline

plt.style.use('seaborn-v0_8-whitegrid')
print('✅ Imports done!')

## 1. Load Raw Data

In [ ]:
ticker = 'RELIANCE.NS'
df_raw = yf.download(ticker, start='2020-01-01', end='2024-12-31', progress=False)
if isinstance(df_raw.columns, pd.MultiIndex):
    df_raw.columns = df_raw.columns.get_level_values(0)

print(f'Raw shape: {df_raw.shape}')
df_raw.head(3)

## 2. Add Technical Indicators — Step by Step

In [ ]:
df = df_raw.copy()
close = df['Close']
high  = df['High']
low   = df['Low']

# ── RSI ──────────────────────────────────────────
df['RSI'] = ta.momentum.RSIIndicator(close=close, window=14).rsi()
print(f'RSI range: {df["RSI"].min():.1f} to {df["RSI"].max():.1f}  (should be 0-100)')

# ── MACD ─────────────────────────────────────────
macd = ta.trend.MACD(close=close, window_slow=26, window_fast=12, window_sign=9)
df['MACD']        = macd.macd()
df['MACD_Signal'] = macd.macd_signal()
df['MACD_Diff']   = macd.macd_diff()
print(f'MACD NaN count: {df["MACD"].isna().sum()} (first ~26 rows expected)')

# ── Bollinger Bands ───────────────────────────────
bb = ta.volatility.BollingerBands(close=close, window=20, window_dev=2)
df['BB_High']  = bb.bollinger_hband()
df['BB_Low']   = bb.bollinger_lband()
df['BB_Width'] = bb.bollinger_wband()
df['BB_Pct']   = bb.bollinger_pband()

# ── EMA ──────────────────────────────────────────
df['EMA_9']    = ta.trend.EMAIndicator(close=close, window=9).ema_indicator()
df['EMA_21']   = ta.trend.EMAIndicator(close=close, window=21).ema_indicator()
df['EMA_Cross'] = df['EMA_9'] - df['EMA_21']

# ── ATR ───────────────────────────────────────────
df['ATR'] = ta.volatility.AverageTrueRange(high=high, low=low, close=close, window=14).average_true_range()

# ── OBV ───────────────────────────────────────────
df['OBV'] = ta.volume.OnBalanceVolumeIndicator(close=close, volume=df['Volume']).on_balance_volume()

# ── Stochastic ────────────────────────────────────
stoch = ta.momentum.StochasticOscillator(high=high, low=low, close=close, window=14, smooth_window=3)
df['Stoch_K'] = stoch.stoch()
df['Stoch_D'] = stoch.stoch_signal()

# ── ROC ───────────────────────────────────────────
df['ROC'] = ta.momentum.ROCIndicator(close=close, window=10).roc()

# ── Price-based ───────────────────────────────────
df['Daily_Return']  = close.pct_change()
df['Price_Range']   = (high - low) / close
df['Close_to_Open'] = (close - df['Open']) / df['Open']

print(f'\nTotal features added: {len(df.columns) - 5}')  # minus OHLCV
print(f'Columns: {list(df.columns)}')

## 3. Label Creation Explained

In [ ]:
# Forward 5-day return
n_days = 5
df['Forward_Return'] = df['Close'].shift(-n_days) / df['Close'] - 1

# Visualize threshold
fwd = df['Forward_Return'].dropna() * 100

fig, ax = plt.subplots(figsize=(12, 4))
ax.hist(fwd, bins=80, color='#2563EB', alpha=0.7, edgecolor='white')
ax.axvline(1,  color='#059669', linestyle='--', lw=2, label='UP threshold (+1%)')
ax.axvline(-1, color='#DC2626', linestyle='--', lw=2, label='DOWN threshold (-1%)')
ax.axvline(0,  color='black',   linestyle='-',  lw=1, alpha=0.4)

up_pct      = (fwd > 1).sum()  / len(fwd) * 100
down_pct    = (fwd < -1).sum() / len(fwd) * 100
neutral_pct = 100 - up_pct - down_pct

ax.set_title(f'5-Day Forward Returns Distribution\n'
             f'UP: {up_pct:.1f}%  |  NEUTRAL: {neutral_pct:.1f}%  |  DOWN: {down_pct:.1f}%',
             fontsize=12, fontweight='bold')
ax.set_xlabel('5-Day Forward Return (%)')
ax.set_ylabel('Frequency')
ax.legend()
plt.tight_layout()
plt.show()

print('Labels explanation:')
print(f'  UP (1):      forward return > +1%  →  {up_pct:.1f}% of samples')
print(f'  NEUTRAL (0): -1% <= return <= +1%  →  {neutral_pct:.1f}% of samples')
print(f'  DOWN (-1):   forward return < -1%  →  {down_pct:.1f}% of samples')

## 4. Sentiment Feature Demo

In [ ]:
# Show how VADER scores financial headlines
test_headlines = [
    'Reliance Industries posts record quarterly profit, beats all estimates',
    'RIL shares crash 8% after disappointing revenue miss',
    'Reliance announces major 5G expansion across India',
    'Market remains cautious ahead of RIL quarterly results',
    'Reliance faces regulatory scrutiny over pricing practices',
    'RIL dividend announcement boosts investor confidence',
]

results = []
for h in test_headlines:
    score = score_headline(h)
    label = 'BULLISH 📈' if score > 0.05 else ('BEARISH 📉' if score < -0.05 else 'NEUTRAL ➡️')
    results.append({'Headline': h[:55]+'...', 'Score': score, 'Sentiment': label})

results_df = pd.DataFrame(results)

# Color coded display
def color_score(val):
    if val > 0.05:  return 'background-color: #D1FAE5'
    if val < -0.05: return 'background-color: #FEE2E2'
    return 'background-color: #FEF3C7'

results_df.style.applymap(color_score, subset=['Score'])

## 5. Final Feature Matrix

In [ ]:
from features import add_technical_indicators, create_labels, prepare_feature_matrix

df_final = add_technical_indicators(df_raw.copy())
df_final = create_labels(df_final, n_days=5)
df_final['Sentiment'] = 0.0  # placeholder — real values from scraper
df_final = prepare_feature_matrix(df_final)

print(f'✅ Final Feature Matrix:')
print(f'   Shape:    {df_final.shape}')
print(f'   Features: {len(df_final.columns) - 1}')
print(f'   Samples:  {len(df_final)}')
print(f'\nFeature names:')
feature_cols = [c for c in df_final.columns if c != 'Label']
for i, col in enumerate(feature_cols, 1):
    print(f'  {i:>2}. {col}')

print(f'\nFirst 3 rows of feature matrix:')
df_final.head(3)

In [ ]:
# Feature stats
print('📊 Feature Statistics (min/max/mean):')
stats = df_final.describe().T[['min', 'max', 'mean', 'std']].round(3)
stats

## 6. Summary

In [ ]:
print('='*50)
print('   🔧 FEATURE ENGINEERING SUMMARY')
print('='*50)
print(f'  Total features built: {len(feature_cols)}')
print()
print('  Feature Groups:')
print('  📈 Momentum:   RSI, MACD, MACD_Signal, MACD_Diff, ROC, Stoch_K, Stoch_D')
print('  📉 Trend:      EMA_9, EMA_21, EMA_Cross')
print('  📊 Volatility: BB_High, BB_Low, BB_Width, BB_Pct, ATR, Price_Range')
print('  📦 Volume:     OBV')
print('  💹 Price:      Daily_Return, Close_to_Open')
print('  📰 Sentiment:  Sentiment (VADER score -1 to +1)')
print()
print('  ➡️  Next: Model Training & Comparison (Notebook 3)')
print('='*50)